In [2]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys
sys.path.append(r"C:\Users\User\RR_model")




In [4]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))

📁 Project root added: C:\Users\MAFC4709\Python_work\RR_model_1911
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000002299EF0DB40>, origin='C:\\Users\\MAFC4709\\Python_work\\RR_model_1911\\src\\__init__.py', submodule_search_locations=['C:\\Users\\MAFC4709\\Python_work\\RR_model_1911\\src'])


In [5]:
# ===========================================================
# 📘 Roll Rate Demo v7 - Sử dụng dữ liệu thật từ Oracle hoặc SQL file
# ===========================================================

from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob

from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

# --- 2️⃣ Load dữ liệu thực tế ---
#sql_path = r"C:/Users/MAFC4709/Python_work/RR_model/sql/uns2.sql"

#df = load_data(sql_path)
df = load_data("C:/Users/MAFC4709/Python_work/RR_model_1911/data/parquet/LGD2.parquet")
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))



📦 Loading Parquet from: C:\Users\MAFC4709\Python_work\RR_model_1911\data\parquet\LGD2.parquet
✅ Loaded 944,702 rows from LGD2.parquet
ℹ️ Added default column PRODUCT_TYPE = 'A'


In [6]:
import pandas as pd

################################################################################
# CONFIG
################################################################################

PRICE_RATE = 0.0002      # 0.02%
CUTOFF_MIN_DATE = "2023-07-01"

MOB_BUCKETS = [
    (0,6),
    (7,12),
    (13,24),
    (25,36),
    (37,999)
]

################################################################################
# Utility functions
################################################################################

def months_between(d1, d2):
    return (d1.year - d2.year) * 12 + (d1.month - d2.month)

def assign_mob_bucket(mob):
    for low, high in MOB_BUCKETS:
        if low <= mob <= high:
            return f"{low}-{high}"
    return "NA"

def get_product_type(row):
    """
    RULE:
    - Nếu PRODUCT_2 == POS_LOAN  → dùng PRODUCT
    - Ngược lại → dùng PRODUCT_2
    """
    if str(row["PRODUCT_2"]).upper() == "POS_LOAN":
        return str(row["PRODUCT"]).upper()
    return str(row["PRODUCT_2"]).upper()

################################################################################
# PREPROCESS
################################################################################

df = df.copy()
df["CUTOFF_DATE_M0"] = pd.to_datetime(df["CUTOFF_DATE_M0"])
df["DISBURSAL_DATE"] = pd.to_datetime(df["DISBURSAL_DATE"])

df = df[df["CUTOFF_DATE_M0"] >= CUTOFF_MIN_DATE].copy()

# PRODUCT TYPE + SEGMENT
df["PRODUCT_TYPE"] = df.apply(get_product_type, axis=1)
df["PRODUCT_SEGMENT"] = df["PRODUCT_TYPE"]

# EAD DEFAULT & PRICE
df["EAD_default"] = df["M0_POS"]
df["price"] = 0.0
df.loc[df["FLAG_SOLDOUT"] == 1, "price"] = df["EAD_default"] * PRICE_RATE

# MOB
df["MOB_default"] = df.apply(
    lambda r: months_between(r["CUTOFF_DATE_M0"], r["DISBURSAL_DATE"]), axis=1
)
df["MOB_BUCKET"] = df["MOB_default"].apply(assign_mob_bucket)

################################################################################
# Function to compute LGD for RW
################################################################################

def compute_lgd_for_rw(df, RW):

    # 1. Only loans with valid POS at RW
    df_rw = df[~df[f"M{RW}_POS"].isna()].copy()

    # 2. EAD at RW
    df_rw[f"EAD_{RW}"] = df_rw[f"M{RW}_POS"]

    # 3. EAD remaining
    df_rw[f"EAD_rem_{RW}"] = df_rw[f"EAD_{RW}"]

    # Sold-out → EAD_remaining = EAD_default - price
    df_rw.loc[df_rw["FLAG_SOLDOUT"] == 1, f"EAD_rem_{RW}"] = (
        df_rw["EAD_default"] - df_rw["price"]
    )

    # 4. LGD loan-level
    df_rw[f"LGD_{RW}"] = df_rw[f"EAD_rem_{RW}"] / df_rw["EAD_default"]

    # 5. GROUP summary
    lookup = (
        df_rw.groupby(["PRODUCT_SEGMENT", "MOB_BUCKET"])
             .agg(
                 LGD_POINT       = (f"LGD_{RW}", "mean"),
                 N_LOANS         = ("AGREEMENT_ID", "nunique"),
                 EAD_DEFAULT_SUM = ("EAD_default", "sum"),
                 EAD_RW_SUM      = (f"EAD_{RW}", "sum"),
                 EAD_REMAIN_SUM  = (f"EAD_rem_{RW}", "sum")
             )
             .reset_index()
    )

    lookup["RW_MONTH"] = RW
    return df_rw, lookup


################################################################################
# COMPUTE RW 12 / 18 / 24
################################################################################

loan12, lookup12 = compute_lgd_for_rw(df, 12)
loan18, lookup18 = compute_lgd_for_rw(df, 18)
loan24, lookup24 = compute_lgd_for_rw(df, 24)

################################################################################
# MERGE LOOKUP INTO ONE TABLE (FULL OUTER JOIN)
################################################################################

lookup_all = lookup12.merge(
    lookup18,
    on=["PRODUCT_SEGMENT","MOB_BUCKET"],
    how="outer",
    suffixes=("_12","_18")
)

lookup24 = lookup24.rename(columns={
    "LGD_POINT": "LGD_POINT_24",
    "N_LOANS": "N_LOANS_24",
    "EAD_DEFAULT_SUM": "EAD_DEFAULT_SUM_24",
    "EAD_RW_SUM": "EAD_RW_SUM_24",
    "EAD_REMAIN_SUM": "EAD_REMAIN_SUM_24"
})

lookup_all = lookup_all.merge(
    lookup24,
    on=["PRODUCT_SEGMENT","MOB_BUCKET"],
    how="outer"
)

################################################################################
# FINAL SELECT COLUMNS
################################################################################

lookup_all = lookup_all[
    [
        "PRODUCT_SEGMENT", "MOB_BUCKET",

        # LGD
        "LGD_POINT_12", "LGD_POINT_18", "LGD_POINT_24",

        # N loans
        "N_LOANS_12", "N_LOANS_18", "N_LOANS_24",

        # EAD default
        "EAD_DEFAULT_SUM_12", "EAD_DEFAULT_SUM_18", "EAD_DEFAULT_SUM_24",

        # POS tại RW
        "EAD_RW_SUM_12","EAD_RW_SUM_18","EAD_RW_SUM_24",

        # EAD remaining (after adjustments)
        "EAD_REMAIN_SUM_12","EAD_REMAIN_SUM_18","EAD_REMAIN_SUM_24",
    ]
]

################################################################################
# SAVE OUTPUT
################################################################################

lookup12.to_excel("LGD_lookup_RW12.xlsx", index=False)
lookup18.to_excel("LGD_lookup_RW18.xlsx", index=False)
lookup24.to_excel("LGD_lookup_RW24.xlsx", index=False)
lookup_all.to_excel("LGD_lookup_all.xlsx", index=False)

print("✓ DONE — Created:")
print("  LGD_lookup_RW12.xlsx")
print("  LGD_lookup_RW18.xlsx")
print("  LGD_lookup_RW24.xlsx")
print("  LGD_lookup_all.xlsx")


✓ DONE — Created:
  LGD_lookup_RW12.xlsx
  LGD_lookup_RW18.xlsx
  LGD_lookup_RW24.xlsx
  LGD_lookup_all.xlsx


In [7]:
import pandas as pd

# Load LGD base (output của mô hình RW18)
lgd_base = lookup_all[[
    "PRODUCT_SEGMENT",
    "MOB_BUCKET",
    "LGD_POINT_18"
]].copy()

lgd_base.rename(columns={"LGD_POINT_18": "LGD_BASE"}, inplace=True)

#############################################
# CONFIG GDP & PARAMETERS
#############################################
g_normal = 6.5        # GDP “bình thường” (%)
beta = 0.01           # 1% LGD increase per 1% GDP shock

# GDP forecast theo scenario
scenarios = [
    {"SCENARIO": "Base",    "GDP_YOY": 6.0},
    {"SCENARIO": "Down",    "GDP_YOY": 4.0},
    {"SCENARIO": "Severe",  "GDP_YOY": 1.0},
    {"SCENARIO": "Upside",  "GDP_YOY": 7.0},   # <-- thêm trường hợp GDP tăng 1%
]

#############################################
# FUNCTION: compute LGD_adjusted for scenarios
#############################################

rows = []
for sc in scenarios:
    shock = g_normal - sc["GDP_YOY"]
    factor = 1 + beta * shock

    df_temp = lgd_base.copy()
    df_temp["SCENARIO"] = sc["SCENARIO"]
    df_temp["GDP_YOY"] = sc["GDP_YOY"]
    df_temp["GDP_SHOCK"] = shock
    df_temp["LGD_FACTOR"] = factor
    df_temp["LGD_ADJ"] = df_temp["LGD_BASE"] * factor

    rows.append(df_temp)

LGD_scenario = pd.concat(rows, ignore_index=True)

# Save file
LGD_scenario.to_excel("LGD_scenario_GDP_adjusted.xlsx", index=False)

print("Done! File created: LGD_scenario_GDP_adjusted.xlsx")


Done! File created: LGD_scenario_GDP_adjusted.xlsx
